# Multivariate QuasiCopula GWAS with Mixed Marginals

Here we adopt the variance component model framework

$$\mathbf{\Gamma}_i(\mathbf{\theta}) = \sum_{k=1}^m \theta_k\mathbf{V}_{ik}, \quad \theta_k \ge 0$$

In [2]:
using Revise
using DataFrames, Random, GLM, QuasiCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using ToeplitzMatrices
using BenchmarkTools
using SnpArrays
using Statistics
using StatsBase

BLAS.set_num_threads(1)
Threads.nthreads()

function simulate_random_snparray(s::Union{String, UndefInitializer}, n::Int64,
    p::Int64; mafs::Vector{Float64}=zeros(Float64, p), min_ma::Int = 5)

    #first simulate a random {0, 1, 2} matrix with each SNP drawn from Binomial(2, r[i])
    A1 = BitArray(undef, n, p) 
    A2 = BitArray(undef, n, p) 
    for j in 1:p
        minor_alleles = 0
        maf = 0
        while minor_alleles <= min_ma
            maf = 0.5rand()
            for i in 1:n
                A1[i, j] = rand(Bernoulli(maf))
                A2[i, j] = rand(Bernoulli(maf))
            end
            minor_alleles = sum(view(A1, :, j)) + sum(view(A2, :, j))
        end
        mafs[j] = maf
    end

    #fill the SnpArray with the corresponding x_tmp entry
    return _make_snparray(s, A1, A2)
end

function _make_snparray(s::Union{String, UndefInitializer}, A1::BitArray, A2::BitArray)
    n, p = size(A1)
    x = SnpArray(s, n, p)
    for i in 1:(n*p)
        c = A1[i] + A2[i]
        if c == 0
            x[i] = 0x00
        elseif c == 1
            x[i] = 0x02
        elseif c == 2
            x[i] = 0x03
        else
            throw(MissingException("matrix shouldn't have missing values!"))
        end
    end
    return x
end

_make_snparray (generic function with 1 method)

## Simulate data

Given $n$ independent samples, we simulate phenotypes from 
$$\mathbf{y}_i \sim QC(\mathbf{\Gamma}, f_1, ..., f_d)$$

In [3]:
# simulate data
p = 3    # number of fixed effects, including intercept
m = 2    # number of variance componentsac
n = 5000 # number of sample
d = 4    # number of phenotypes per sample
q = 1000 # number of SNPs
k = 0    # number of causal SNPs

# sample d marginal distributions for each phenotype within samples
Random.seed!(1234)
possible_distributions = [Bernoulli, Poisson, Normal]
vecdist = rand(possible_distributions, d)
# vecdist = [Poisson, Bernoulli, Bernoulli] # this derivative test is fine
# vecdist = [Bernoulli, Bernoulli, Poisson] # this derivative test is wrong everywhere
veclink = [canonicallink(vecdist[j]()) for j in 1:d]

# simulate nongenetic coefficient and variance component params
Random.seed!(2022)
Btrue = rand(Uniform(-0.5, 0.5), p, d)
θtrue = fill(0.4, m)
V1 = ones(d, d)
V2 = Matrix(I, d, d)
Γ = m == 1 ? θtrue[1] * V1 : θtrue[1] * V1 + θtrue[2] * V2

# simulate non-genetic design matrix
Random.seed!(2022)
X = [ones(n) randn(n, p - 1)]

# simulate random SnpArray with q SNPs and randomly choose k SNPs to be causal
Random.seed!(2022)
G = simulate_random_snparray(undef, n, q)
Gfloat = convert(Matrix{Float64}, G, center=true, scale=true)
γtrue = zeros(q, d)
causal_snps = sample(1:q, k, replace=false) |> sort
for j in 1:d
    γtrue[causal_snps, j] .= rand([-1, 1], k)
end

# sample phenotypes
Y = zeros(n, d)
y = Vector{Float64}(undef, d)
for i in 1:n
    Xi = X[i, :]
    Gi = Gfloat[i, :]
    η = Btrue' * Xi + γtrue' * Gi
    vecd_tmp = Vector{UnivariateDistribution}(undef, d)
    for j in 1:d
        dist = vecdist[j]
        μj = GLM.linkinv(canonicallink(dist()), η[j])
        vecd_tmp[j] = dist(μj)
    end
    multivariate_dist = MultivariateMix(vecd_tmp, Γ)
    res = Vector{Float64}(undef, d)
    rand(multivariate_dist, y, res)
    Y[i, :] .= y
end

# form model
V = m == 1 ? [V1] : [V1, V2]
qc_model = MultivariateCopulaVCModel(Y, X, V, vecdist, veclink);

What does phenotypes look like?

In [4]:
Y

5000×4 Matrix{Float64}:
 0.0  0.0  0.0  -0.80051
 1.0  5.0  1.0  -1.02321
 0.0  0.0  1.0  -0.433156
 1.0  1.0  0.0  -0.624015
 0.0  0.0  1.0   0.0960992
 0.0  2.0  0.0   1.22445
 0.0  2.0  1.0  -0.0417102
 0.0  2.0  1.0   1.44549
 0.0  0.0  0.0  -1.74432
 1.0  1.0  0.0  -1.03616
 1.0  2.0  1.0  -0.0715883
 0.0  2.0  1.0  -1.30745
 0.0  0.0  0.0  -1.03096
 ⋮              
 0.0  0.0  0.0  -1.55683
 0.0  0.0  0.0   0.866433
 0.0  3.0  1.0   0.261847
 1.0  3.0  1.0  -1.37618
 0.0  3.0  1.0  -0.2299
 0.0  0.0  0.0  -2.05345
 1.0  2.0  1.0  -1.7245
 1.0  2.0  0.0   2.02739
 1.0  0.0  1.0   0.495043
 0.0  1.0  1.0  -0.791913
 0.0  0.0  1.0  -1.76057
 0.0  1.0  0.0  -0.411001

What does our covariates look like?

In [5]:
X

5000×3 Matrix{Float64}:
 1.0  -0.308648    1.70162
 1.0   1.67671    -0.548034
 1.0  -0.347153    0.736227
 1.0   0.818666   -2.16009
 1.0  -1.71753    -0.273745
 1.0  -0.238934    0.942883
 1.0   0.701932    1.02868
 1.0  -0.166138   -0.278824
 1.0  -0.609614    0.289359
 1.0   0.68791     0.209478
 1.0   0.0342303  -0.543192
 1.0  -0.479078   -0.865401
 1.0  -1.63537     0.348029
 ⋮                
 1.0   0.200555    1.14607
 1.0  -0.205806    1.98172
 1.0   1.17812     0.307879
 1.0   1.60549     0.817788
 1.0   1.63509    -0.960082
 1.0  -0.446096   -1.0502
 1.0   0.632009   -0.335688
 1.0   0.589777   -1.92135
 1.0  -0.542628    1.68057
 1.0  -0.779274    0.6376
 1.0  -1.19111    -1.3064
 1.0   0.505272    1.11117

Support for the phenotypes

In [6]:
vecdist

4-element Vector{UnionAll}:
 Bernoulli
 Poisson
 Bernoulli
 Normal

In [7]:
# true Γ scaled to correlation matrix
cov2cor(Γ, diag(Γ))

4×4 Matrix{Float64}:
 1.0    0.625  0.625  0.625
 0.625  1.0    0.625  0.625
 0.625  0.625  1.0    0.625
 0.625  0.625  0.625  1.0

In [8]:
# empirical correlation of Y
Statistics.cor(Y)

4×4 Matrix{Float64}:
 1.0       0.161353   0.141082   0.129986
 0.161353  1.0        0.0641498  0.0698481
 0.141082  0.0641498  1.0        0.135562
 0.129986  0.0698481  0.135562   1.0

## Fit Null model

TODO: 

+ Initializing model parameters

In [9]:
@time optm = QuasiCopula.fit!(qc_model,
    Ipopt.IpoptSolver(
        print_level = 5, 
        tol = 10^-6, 
        max_iter = 100,
        accept_after_max_steps = 10,
        warm_start_init_point="yes", 
        limited_memory_max_history = 6, # default value
        hessian_approximation = "limited-memory",
        derivative_test="first-order"
    )
);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.


No errors detected by derivative checker.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       14
                     variables with only lower bounds:        2
                variables with lower and upper bounds

In [10]:
@show qc_model.∇vecB
@show qc_model.∇θ;

qc_model.∇vecB = [-2.555665467918189e-6, -2.666047984573039e-6, -4.23217445452706e-6, -5.6345981152006175e-6, 1.6536963753754907e-5, 1.5208381801024018e-5, 7.784947875233428e-6, 5.083556454399396e-7, 2.1021208863603036e-6, -5.5324851026272714e-5, -3.0735902118744773e-5, -1.9942155756624835e-5]
qc_model.∇θ = [-3.8057636611577017e-6, -5.710495082411615e-6]


In [11]:
[vec(qc_model.B) vec(Btrue)]

12×2 Matrix{Float64}:
 -0.276263    -0.280755
  0.16857      0.167603
 -0.28159     -0.330996
 -0.0478524   -0.0469412
  0.494938     0.483384
 -0.279155    -0.269155
  0.13693      0.144784
 -0.398999    -0.39274
 -0.00312567   0.0260867
 -0.235556    -0.234534
 -0.100403    -0.0969734
  0.08936      0.0955352

In [12]:
[qc_model.θ θtrue]

2×2 Matrix{Float64}:
 0.453502  0.4
 0.412442  0.4

## Score test